In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
xx = QQ['xx'].0

In [3]:
Q.<alpha> = NumberField(xx^2+3)

In [4]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [5]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [6]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [7]:
line = Line([y, z])

In [8]:
eqn_SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )
sing_cubics_SE1 = P(sing_cubics.subs({b:-(c*c+e*f)/c }))

In [9]:
eqn_SE3 = P(2*eqn_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}) )
sing_cubics_SE3 = P(sing_cubics_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}))

In [10]:
eqn_SE9 = P(eqn_SE3.subs({e : alpha*c}) /c)
sing_cubics_SE9 = P(sing_cubics_SE3.subs({e : alpha*c}))

In [11]:
SE9 = Cubic(eqn_SE9, line, sing_cubics_SE9.factor())

In [12]:
SE9.eckardt_points

[(0, 0, 0, 1),
 (0, 1, 1, 0),
 ((-1/3*alpha), (-1/3*alpha), 1, -1),
 ((1/4*alpha + 1/4)*c^2 + (-1/4*alpha - 1/4)*c*f, c^2, c*f, (-1/4*alpha - 1/4)*c*f + (1/4*alpha + 1/4)*f^2),
 ((1/6*alpha)*c^2 + (-1/6*alpha)*c*f, (1/6*alpha + 1/2)*c^2, (1/6*alpha + 1/2)*c*f, (1/4*alpha + 1/4)*c^2 + (-1/3*alpha - 1/2)*c*f + (1/12*alpha + 1/4)*f^2),
 ((-5/42*alpha - 1/14)*c + (5/42*alpha + 1/14)*f, (2/21*alpha - 1/7)*c + (2/21*alpha - 1/7)*f, (1/7*alpha + 2/7)*c + (1/21*alpha - 4/7)*f, (1/14*alpha - 5/14)*c + (-1/14*alpha + 5/14)*f),
 ((-1/4*alpha + 1/4)*c^2 + (1/4*alpha - 1/4)*c*f, -1/2*c^2 - 1/2*c*f, (1/2*alpha)*c^2 + (-1/2*alpha - 1)*c*f, (-1/2*alpha)*c^2 + (3/4*alpha + 1/4)*c*f + (-1/4*alpha - 1/4)*f^2),
 (0, (1/2*alpha - 1/2)*c, (1/2*alpha - 1/2)*c, (-1/2*alpha - 1/2)*c + (1/2*alpha + 1/2)*f),
 (1/2*c, 1/2*c, (1/2*alpha)*c, -1/2*f)]

In [13]:
%time adm_SE9 = SE9.find_admissible_projectivities()

CPU times: user 6.04 s, sys: 206 ms, total: 6.24 s
Wall time: 1min 17s


In [14]:
len(adm_SE9)

1296

In [15]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)

CPU times: user 1.59 s, sys: 157 ms, total: 1.74 s
Wall time: 5.28 s


In [16]:
len(simm_SE9)

54

In [17]:
G_SE9 = Group(simm_SE9)

In [18]:
G_SE9.get_order()

54

In [19]:
G_SE9.is_abelian()

False

In [20]:
G_SE9.get_divisors_of_order()

[1, 2, 3, 6, 9, 18, 27, 54]

In [23]:
#G_SE9.get_order_of_elements()

In [22]:
#[G_SE9.apply_to(p) for p in SE9.eckardt_points]

NameError: name 'eckardt_points' is not defined